In [1]:
import numpy as np
from scipy.stats import norm
import yfinance as yf
import datetime
from datetime import datetime as dt, timedelta
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


In [2]:
N = norm.cdf


def bsm_call_price(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - np.exp(-r * T) * N(d2)


def bsm_put_price(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * N(-d2) - S * N(-d1)


# Reminder:
# S = stock price
# K = strike price
# r = risk-free rate
# T = time to maturity (365days)
# sigma = volatility $\sigma$


In [15]:
def IV(S, K, r, T, mkt_price, opt_type="C" or "P"):
    dt = 1e-3
    # volatility = input(float("Enter volatility: "))
    volatility = 1.060
    step = 1e-3

    for i in range(1000):
        if opt_type == "C":
            price = bsm_call_price(S, K, r, T, sigma=volatility)
        else:
            price = bsm_put_price(S, K, r, T, sigma=volatility)

        diff = mkt_price - price
        if diff > dt:
            volatility = volatility + step
        elif diff < 0 and abs(diff) > dt:
            volatility = volatility - step
        elif abs(diff) < dt:
            return volatility
    return volatility  # if best value not found


In [16]:
def expected_move_IV(stock, K, r, T, mkt_price, opt_type="P" or "C"):
    future = datetime.datetime.strptime(T, "%Y-%m-%d")
    today = datetime.datetime.today()
    delta = (future - today).days
    t = delta / 365

    ticker = yf.Ticker(stock)
    data = ticker.history()
    last_quote = data["Close"].iloc[-1]

    # calc iv
    iv = IV(last_quote, K, r, t, mkt_price, opt_type)

    # calc expected move
    move = last_quote * (iv) * np.sqrt(t)

    upper_bound = last_quote + round(move, 2)
    lower_bound = last_quote - round(move, 2)

    backdate = dt.now() + timedelta(days=-100)
    date = backdate.strftime("%Y-%m-%d")

    # get historical data
    frame = yf.download(stock, start=date)
    b = pd.date_range(start=date, end=future)
    frame.index.append(b)

    # graph historical performance & expected price range
    fig = px.line(
        frame,
        x=frame.index,
        y=frame["Close"],
        title=f"{stock} Expected Price Range (+/- {round(move, 2)})",
    )
    fig.update_xaxes(range=[date, future])
    fig.add_hline(
        y=upper_bound,
        line=dict(color="red", width=3),
        line_dash="dot",
        annotation_text=upper_bound,
        annotation_position="bottom right",
    )
    fig.add_hline(
        y=lower_bound,
        line=dict(color="red", width=3),
        line_dash="dot",
        annotation_text=lower_bound,
        annotation_position="bottom right",
    )
    fig.update_yaxes(title_text="Stock Price")
    fig.update_xaxes(title_text="Date")
    fig.update_layout(height=800, width=1500, showlegend=True)

    return fig.show()


In [18]:
expected_move_IV('VKTX', 30, 0.0525, '2023-07-23', 2.40, 'C')

[*********************100%***********************]  1 of 1 completed
